__DSC 478__

__Jonathan Lynch__

# Assignment 4 - Part 1

## Recommender Systems

__Import libraries__

In [1]:
import numpy as np
from numpy import *
from numpy import linalg as la

## A: Load in the joke ratings data and the joke text data into appropriate data structures:

In [2]:
%cd "/Users/jonathanlynch/Desktop/DSC 478/jokes"

ratings = np.genfromtxt('modified_jester_data.csv', delimiter=',')

jokes = np.genfromtxt('jokes.csv', delimiter=',',dtype=str)

/Users/jonathanlynch/Desktop/DSC 478/jokes


In [3]:
ratings.shape

(1000, 100)

In [4]:
jokes.shape

(100, 2)

In [5]:
jokes[0][1]  # look at first joke

'A man visits the doctor. The doctor says "I have bad news for you.You have cancer and Alzheimer\'s disease". The man replies "Well thank God I don\'t have cancer!"'

## B: Complete the definition for the function "test".  Use this function to perform evaluation (with 20% test-ratio for each user) comparing MAE results using standard item-based collaborative filtering (based on the rating prediction function "standEst") with results using the SVD-based version of the rating item-based CF (using "svdEst" as the prediction engine):

__Code provided in itemBasedRec.py (from Chap 14 of MLA):__

In [6]:
def ecludSim(inA,inB):
    return 1.0 / (1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5 + 0.5 * corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    num = float(inA.T * inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5 + 0.5 * (num / denom)

def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0: continue
        overLap = nonzero(logical_and(dataMat[:,item]>0, \
                                      dataMat[:,j]>0))[0]
        if len(overLap) == 0: similarity = 0
        else: similarity = simMeas(dataMat[overLap,item], \
                                   dataMat[overLap,j])
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    data=mat(dataMat)
    U,Sigma,VT = la.svd(data)
    Sig4 = mat(eye(4)*Sigma[:4]) #arrange Sig4 into a diagonal matrix
    xformedItems = data.T * U[:,:4] * Sig4.I  #create transformed items
    for j in range(n):
        userRating = data[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

def cross_validate_user(dataMat, user, test_ratio, estMethod, simMeas):
    number_of_items = np.shape(dataMat)[1]
    rated_items_by_user = np.array([i for i in range(number_of_items) if dataMat[user,i]>0])
    test_size = int(test_ratio * len(rated_items_by_user))
    test_indices = np.random.randint(0, len(rated_items_by_user), test_size)
    withheld_items = rated_items_by_user[test_indices]
    original_user_profile = np.copy(dataMat[user])
    dataMat[user, withheld_items] = 0 # so that the withheld test items is not used in the rating estimation below
    error_u = 0.0
    count_u = len(withheld_items)

    # compute absolute error for user u over all test items
    for item in withheld_items:
        # estimate rating on the withheld item
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        error_u = error_u + abs(estimatedScore - original_user_profile[item])

    # restore ratings of the withheld items to the user profile
    for item in withheld_items:
        dataMat[user, item] = original_user_profile[item]

    # return sum of absolute errors and the count of test cases for this user
    return error_u, count_u

__Complete function "test" definition:__

In [7]:
def test(dataMat, test_ratio, estMethod, simMeas):
    'function to iterate over all users, perform cross-validation, and print MAE'
    total_error = 0
    total_count = 0
    for user in range(dataMat.shape[0]):
        error_u, count_u = cross_validate_user(dataMat, user, test_ratio, estMethod, simMeas)
        total_error += error_u
        total_count += count_u
    MAE = total_error/total_count
    print ('Mean Absoloute Error for ',estMethod,' : ', MAE)

__Compare MAE results using standard item-based collaborative filtering vs SVD-based version:__

In [8]:
test(ratings, .2, standEst, pearsSim)

Mean Absoloute Error for  <function standEst at 0x7ff57f873c10>  :  3.692740366748648


The mean absolute error using standard item-based collaborative filtering (based on the rating prediction function "standEst") and a Pearson similarity measure is approximately 3.69.

In [9]:
test(ratings, .2, svdEst, pearsSim)

Mean Absoloute Error for  <function svdEst at 0x7ff57f873ca0>  :  3.636041315959524


The mean absolute error using the SVD-based version of the rating item-based collaborative filtering (based on the function "svdEst") and a Pearson similarity measure is approximately 3.64.

## C: Write a new function "print_most_similar_jokes" which takes the joke ratings data, a query joke id, a parameter k for the number of nearest neighbors, and a similarity metric function, and prints the text of the query joke as well as the texts of the top k most similar jokes based on user ratings:

__Write function "print_most_similar_jokes" to display the k most similar jokes, given a query joke & a similarity metric function:__

In [10]:
def print_most_similar_jokes(data, jokes, joke, k, metric):
    'function to print k most similar jokes using selected similarity metric function'
    print ("Selcted joke: ")
    print()
    print (jokes[joke][1])                          # print selected joke
    print()
    distances = []
    joke_vector = data[:,joke]                      # vector of joke ratings for selected joke
    for i in range(data.shape[1]):
        joke_vectors = data[:,i]                    # vectors of joke ratings for each joke
        dist = metric(joke_vector, joke_vectors)    # distances based on similarity measure
        distances.append(dist)
    distances_array = np.array(distances)
    sorted_distances = distances_array.argsort()  # get sorted indicies of distances_array
    top_indicies = sorted_distances[:k]           # top k indicies
    print("The top %d recommended jokes are: "%(k))
    for indice in top_indicies:
        top_joke = jokes[indice]       # top jokes
        print()
        print(top_joke[1])

__Look at 5 most similar jokes to joke #3, using a Euclidean distance metric function:__

In [11]:
print_most_similar_jokes(ratings, jokes, 3, 5, ecludSim)

Selcted joke: 

Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.

The top 5 recommended jokes are: 

A guy goes into confession and says to the priest "Father I'm 80 years old widower with 11 grandchildren. Last night I met two beautiful flight attendants. They took me home and I made love to both of them. Twice."The priest said: "Well my son when was the last time you were in confession?" "Never Father I'm Jewish." "So then why are you telling me?" "I'm telling everybody."

A guy walks into a bar orders a beer and says to the bartender"Hey I got this great Polish Joke..." The barkeep glares at him and says in a warning tone of voice:"Before you go telling that joke you better know that I'm Polish both bouncers are Polish and so are most of my customers""Okay" says the customer"I'll tell it very slowly."

A group of  managers were given the assignment to measure the height of a flagpole. So they go out to the flagpole with lad